In [6]:
import pandas as pd

# Load the CSV dataset (update filename if different)
df = pd.read_csv("dataset.csv")

# Convert date and time columns to datetime
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'])

# Sort by timestamp to simulate real-time
df = df.sort_values(by='Datetime').reset_index(drop=True)


In [7]:
def baseline_pricing(prev_price, occupancy, capacity, alpha=1.0):
    return round(prev_price + alpha * (occupancy / capacity), 2)


In [8]:
def calculate_demand(row, weights):
    demand = (
        weights['occupancy'] * (row['Occupancy'] / row['Capacity']) +
        weights['queue'] * row.get('QueueLength', 0) -
        weights['traffic'] * {'low': 1, 'average': 2, 'high': 3}.get(row['TrafficConditionNearby'], 1) +
        weights['special'] * row['IsSpecialDay'] +
        weights['vehicle'] * {'bike': 0.5, 'car': 1, 'truck': 1.5, 'cycle': 0.3}.get(row['VehicleType'], 1)
    )
    return demand


In [9]:
def demand_based_price(base_price, demand, lam=0.5):
    # Normalize demand (e.g., scale between 0 and 1)
    norm_demand = min(max(demand / 10, 0), 1)  # Assuming demand max = 10
    price = base_price * (1 + lam * norm_demand)
    return round(min(max(price, base_price * 0.5), base_price * 2), 2)


In [10]:
from geopy.distance import geodesic

def nearby_parking_spaces(current_row, df, radius_km=0.5):
    current_coords = (current_row['Latitude'], current_row['Longitude'])
    nearby = []
    for _, row in df.iterrows():
        coords = (row['Latitude'], row['Longitude'])
        if geodesic(current_coords, coords).km <= radius_km:
            nearby.append(row['ID'])
    return nearby


In [11]:
import time

prices = {}  # ID: last_price
BASE_PRICE = 10
weights = {
    'occupancy': 1.0,
    'queue': 0.5,
    'traffic': 0.3,
    'special': 1.2,
    'vehicle': 0.7,
}

for idx, row in df.iterrows():
    lot_id = row['ID']
    prev_price = prices.get(lot_id, BASE_PRICE)

    # Model 2 Pricing
    demand = calculate_demand(row, weights)
    new_price = demand_based_price(BASE_PRICE, demand)

    prices[lot_id] = new_price
    print(f"Time: {row['Datetime']}, Lot: {lot_id}, Price: ${new_price}")

    time.sleep(0.1)  # simulate stream


Time: 2016-01-11 08:06:00, Lot: 12276, Price: $10.85
Time: 2016-01-11 08:06:00, Lot: 1780, Price: $10.24
Time: 2016-01-11 08:06:00, Lot: 13588, Price: $10.78
Time: 2016-01-11 08:06:00, Lot: 9652, Price: $10.85
Time: 2016-01-11 08:06:00, Lot: 3092, Price: $10.82
Time: 2016-01-11 08:06:00, Lot: 7028, Price: $10.63
Time: 2016-01-11 08:06:00, Lot: 14900, Price: $10.81
Time: 2016-01-11 08:06:00, Lot: 4404, Price: $10.76
Time: 2016-01-11 08:06:00, Lot: 468, Price: $10.49
Time: 2016-01-11 08:06:00, Lot: 5716, Price: $10.98
Time: 2016-01-11 08:06:00, Lot: 5717, Price: $10.81
Time: 2016-01-11 08:06:00, Lot: 16212, Price: $11.05
Time: 2016-01-11 08:06:00, Lot: 8340, Price: $10.83
Time: 2016-01-11 08:06:00, Lot: 17524, Price: $10.61
Time: 2016-01-11 08:06:00, Lot: 10964, Price: $10.73
Time: 2016-01-11 08:20:00, Lot: 7029, Price: $10.89
Time: 2016-01-11 08:26:00, Lot: 14901, Price: $10.82
Time: 2016-01-11 08:26:00, Lot: 17525, Price: $10.97
Time: 2016-01-11 08:26:00, Lot: 3093, Price: $11.05
Time:

In [12]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook

output_notebook()

source = ColumnDataSource(data=dict(time=[], price=[]))

p = figure(title="Real-time Parking Price", x_axis_type='datetime')
p.line(x='time', y='price', source=source)

handle = show(p, notebook_handle=True)

for idx, row in df.iterrows():
    time_val = row['Datetime']
    demand = calculate_demand(row, weights)
    price = demand_based_price(BASE_PRICE, demand)

    new_data = dict(time=[time_val], price=[price])
    source.stream(new_data, rollover=300)
    push_notebook(handle=handle)

    time.sleep(0.1)


Loading BokehJS ...